# Bayesian regression for storm erosion prediction
Joshua Simmons 05/2022

Further ideas:
 - https://nbviewer.org/github/pyro-ppl/numpyro/blob/master/notebooks/source/bayesian_regression.ipynb

In [ ]:
# magic
%load_ext autoreload
%autoreload 2
%pdb 1
%matplotlib inline

## Imports and settings

In [ ]:
import os, sys
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('talk')

from ipywidgets import interactive, fixed, widgets

# MAP regression
import sklearn.linear_model as lm

# pymc3

# import pymc3 as pm

# from pymc3 import HalfCauchy, Model, Normal, sample, HalfNormal

import urllib.request

## Download wave data

In [ ]:
# the most recent data
rawWaveData = pd.read_csv(
    os.path.join(
        '/data',
        'Waverider_buoys_Observations_-_Australia_-_near_real-time.csv'
    ),
    skiprows=16
)
rn_dict = {
    'WHTH': 'Hsig',
    'VDIR': 'Wdir',
    'SPWP': 'Tp'
}
rawWaveData.rename(columns=rn_dict,inplace=True)

rawWaveData = rawWaveData[['TIME','site_name','Hsig','Tp','Wdir']]

wave_data_sorted = {
    _: rawWaveData.query('site_name == "{}"'.format(_)) for _ in rawWaveData.site_name.unique()
}

for _ in wave_data_sorted.keys():
    wave_data_sorted[_] = wave_data_sorted[_].set_index('TIME').sort_index()

In [ ]:
# the most recent data
rawWaveData = pd.read_csv(
    os.path.join(
        '/data',
        'Waverider_buoys_Observations_-_Australia_-_delayed_National_Wave_Archive.csv'
    ),
    skiprows=99
)
rn_dict = {
    'WHTH': 'Hsig',
    'WPDI': 'Wdir',
    'WPPE': 'Tp'
}
rawWaveData.rename(columns=rn_dict,inplace=True)

rawWaveData = rawWaveData[['TIME','site_name','Hsig','Tp','Wdir']]

wave_data_sorted = {
    _: rawWaveData.query('site_name == "{}"'.format(_)) for _ in rawWaveData.site_name.unique()
}

for _ in wave_data_sorted.keys():
    wave_data_sorted[_] = wave_data_sorted[_].set_index('TIME').sort_index()


In [ ]:
rawWaveData.head()

In [ ]:
wave_data_sorted['Sydney']['Hsig'].plot()

In [ ]:
rawWaveData = pd.read_csv('combined_era_data_-34.0_151.5.csv',index_col=0,parse_dates=True)
rawWaveData

## Download shoreline data

In [ ]:
transectName = 'aus0206-0005'
csSource = 'http://coastsat.wrl.unsw.edu.au/time-series/{}/'
tmpLoc =  "working_data.csv"
download = False

if download:
    urllib.request.urlretrieve(csSource.format(transectName), tmpLoc)

rawShlData = pd.read_csv(tmpLoc,parse_dates=True,index_col=0,header=None)
rawShlData.index = pd.to_datetime(rawShlData.index,utc=True)
rawShlData.columns = ['Shoreline']
rawShlData.index.name = 'Date'
# shlData.plot()

diffShlData = rawShlData.diff()
diffShlData.columns = ['dShl'] 

sns.histplot(diffShlData.dropna())
None

## Convert to storm dataset

In [ ]:
def calculate_wave_energy(waveIn):
    rho = 1025 # kg/m^3
    g = 9.81 # m/s^2
    # this needs to be more robust
    dt = (waveIn.index[1] - waveIn.index[0]).seconds/3600
    stormCriteria = waveIn['Hsig']>3.0 #m
    energy = np.sum(waveIn.loc[stormCriteria,'Hsig']**2)*rho*g*dt*(1/16)
    return energy

In [ ]:
# Prepare the diffShlData df
shlData = rawShlData.diff().copy()
shlData['postDate'] = shlData.index
shlData.loc[shlData.index[1:],'preDate'] = shlData.index[:-1]
shlData.drop(shlData.index[0],inplace=True)
shlData.index = ['Storm_{0:04.0f}'.format(_) for _ in range(shlData.shape[0])]
shlData

In [ ]:
# split the wave data according to the shoreline data
waveData = {}
for thisStorm in shlData.index:
    waveData[thisStorm] = rawWaveData.loc[shlData.loc[thisStorm,'preDate']:shlData.loc[thisStorm,'postDate']]
    shlData.loc[thisStorm,'E'] = calculate_wave_energy(waveData[thisStorm])
waveData[thisStorm]
shlData

In [ ]:
# constrain shlData by adding consective shoreline movements
# print(shlData)
shlData['zeroCross'] = np.sign(shlData['Shoreline']).diff().ne(0).astype(int)
shlData['zeroCross'] = shlData['zeroCross'].cumsum()
groupedVals = shlData.groupby(by='zeroCross',as_index=False).sum()
groupedVals.index = shlData.index[[np.where(shlData['zeroCross'] == _)[0][0] for _ in groupedVals['zeroCross']]]
shlData['E'] = groupedVals['E']
shlData['Shoreline'] = groupedVals['Shoreline']
shlData.drop_duplicates(subset=['zeroCross'],inplace=True)
shlData

## Plot the data

In [ ]:
# clean the data
plotData = shlData.dropna().copy()
plotData = plotData.loc[(plotData['E']>250000) & (plotData['Shoreline']<-5)]
plotData = plotData.drop(plotData.index[(plotData['E']>600000) & (plotData['Shoreline']>-20)])

x, y = plotData['E'].values, -plotData['Shoreline'].values

# and trasnform to logspace
x_log = np.log(x)
y_log = np.log(y)

log_scale=False
fig = plt.figure(figsize=(7, 5))
ax1 = fig.add_subplot(111)

sns.scatterplot(x='E',y='Shoreline',hue='preDate',data=plotData,ax=ax1)

ax1.set_xlabel('E')
ax1.set_ylabel('dShl')
ax1.invert_yaxis()
# change axes to log - log scale
if log_scale:
    plt.xscale('log')
    plt.yscale('log')
plt.title('Shoreline change from satelite')
ax1.get_legend().remove()
# plt.legend(loc='center', bbox_to_anchor=(1.5,0.5))
plt.show()

## Generate toy data

In [ ]:
# Generate data that follows a power law distribution
# y = a * x^b + error
#defaults
powa = 0.05
constb = 1.5
err_var = 0.15
hs_scale = 0
# Plot x, y data

def generate_data(powa, constb, err_var, hs_scale, n_points, log_scale=False):
    print('Choose a and b in y = a * x^b')
    x = np.around(np.random.uniform(0, 10, n_points),decimals=2)

    # generate the error
    err = np.random.normal(0, err_var, len(x))
    # add a homoscedastic error
    err += (err * x) * hs_scale


    y = (powa * x**constb) + err

    plt.plot(x, y, 'o',label='Data')
    xReal = np.linspace(0,10,100)
    yReal = powa * xReal**constb
    plt.plot(xReal,yReal,'r-',label='True function')

    plt.xlabel('x')
    plt.ylabel('y')
    # change axes to log - log scale
    if log_scale:
        plt.xscale('log')
        plt.yscale('log')
    plt.title('Power Law Distribution')
    plt.legend(loc='center', bbox_to_anchor=(1.1,0.5))
    plt.show()

    return x, y

result = interactive(
    generate_data,
    powa = widgets.FloatText(value=powa,description='a',step=0.05), 
    constb = widgets.FloatText(value=constb,description='b',step=0.5),
    err_var = widgets.FloatText(value=err_var,description='variance',step=0.05),
    hs_scale = widgets.FloatText(value=hs_scale,description='homoscedasticity',step=0.05),
    n_points = widgets.IntSlider(value=5,min=5,max=100,step=5,description='n_points'),
    log_scale=False
)

display(result)



In [ ]:
x, y = result.result
print('x: {}\ny: {}'.format(x.shape,y.shape))

# and trasnform to logspace
x_log = np.log(x)
y_log = np.log(y)

## Setup plotting functions

In [ ]:
def draw_fit(xObs,yObs,xPred,yPred,ySample=None,yMAP=None,**kwargs):
    fig = plt.figure(figsize=(7, 5))
    ax1 = fig.add_subplot(111)

    # if kwargs.get('errBands',False):
    #     ax1.fill_between(xSample, predyMean - 1.96*predyStd, predyMean + 1.96*predyStd, alpha=0.2,label='95% CI')
    #     ax1.plot(xSample, sampleyMean[:,0],label='Samples')
    #     ax1.plot(xSample, sampleyMean[:,1:])
    # else:
    #     ax1.plot(xSample, sampleyMean)#,label='Samples')

    if not ySample is None:
        ax1.plot(xPred, ySample[0,:].T,'-',color='xkcd:light grey',alpha=0.4,label='Samples')
        ax1.plot(xPred, ySample[1:,:].T,'-',color='xkcd:light grey',alpha=0.4)

    if not yMAP is None:
        ax1.plot(xPred, yMAP.T,'-',color='xkcd:red',label='MAP fit')

    # # plot mean
    # ax1.plot(xSample, predyMean, 'k', label='Mean')

    # if not y is None:
    ax1.plot(xObs, yObs, 'o',color='xkcd:dark grey',label='Observed')
    ax1.plot(xPred, yPred, 'C0', label='Predicted')
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')

    if kwargs.get('log_scale',False):
        ax1.set_xscale('log')
        ax1.set_yscale('log')

    ax1.set_title(kwargs.get('title','Power Law Distribution Fit'))
    # place legend outside the plot
    plt.legend(loc='center', bbox_to_anchor=(1.2, 0.5))
    plt.show()
    return ax1

## Fit a Bayesian Linear Regression in logspace - MAP

In [ ]:
# fit a linear model to the data
model = lm.LinearRegression()
model.fit(x_log.reshape(-1,1),y_log)

sample_x = np.linspace(x.min(),x.max(),101)

y_pred = model.predict(np.log(sample_x).reshape(-1,1))

draw_fit(x,y,sample_x,np.exp(y_pred),title='MAP Fit')

draw_fit(x,y,sample_x,np.exp(y_pred),log_scale=True,title='MAP Fit Log-Log')

None

## Fit with pymc3

In [ ]:
with Model() as model:  # model specifications in PyMC3 are wrapped in a with-statement
    # Define priors
    sigma = HalfCauchy("sigma", beta=10, testval=1.0)
    intercept = Normal("Intercept",0,sigma=10)
    x_coeff = Normal("x", 0, sigma=10)

    # Define likelihood
    likelihood = Normal("y", mu=intercept + x_coeff * x_log, sigma=sigma, observed=y_log)

    # Inference!
    # draw 3000 posterior samples using NUTS sampling
    trace = sample(10000, return_inferencedata=True)

In [ ]:
import arviz as az
az.plot_trace(trace, figsize=(10, 7));

In [ ]:
nSamples = 100

# generate draws from the posterior
eval_lm = lambda x, sample: sample["Intercept"] + sample["x"] * x
if not isinstance(trace, list):
    trace = trace.posterior.to_dataframe().to_dict(orient="records")
ySample = np.zeros((nSamples, len(sample_x)))
for ii, rand_loc in enumerate(np.random.randint(0, len(trace), nSamples)):
    rand_sample = trace[rand_loc]
    tmpSample = eval_lm(np.log(sample_x), rand_sample)
    ySample[ii,:] = tmpSample

mapParams = pm.find_MAP(model=model)
yMAP = eval_lm(np.log(sample_x), mapParams)

draw_fit(
    x,y,
    sample_x,np.full(sample_x.shape, np.nan),
    ySample=np.exp(ySample),yMAP=np.exp(yMAP),
    title='Fit with uncertainty')

draw_fit(
    x,y,
    sample_x,np.full(sample_x.shape, np.nan),
    ySample=np.exp(ySample),yMAP=np.exp(yMAP),
    log_scale=True,title='Log-Log fit with uncertainty')

None
